# Análise de compras conjuntas por Frequent Pattern Growth (FPGrowth)

## Importação de bibliotecas

In [1]:
from pyspark.ml.fpm import FPGrowth
from pyspark import SparkContext, SQLContext
from pyspark.sql import SparkSession

import pandas as pd

## Leitura de dados

De `all_orders.csv` obtemos os identificadores de consumidor, os quais podemos cruzar com os dados de `sold_products.csv`, obtendo uma lista de compras por cliente

In [2]:
all_orders = pd.read_csv('../datasets/trusted/all_orders.csv')
all_orders.count()

Unnamed: 0                       99441
order_id                         99441
customer_unique_id               99441
customer_city                    99441
customer_state                   99441
order_approved_at                99281
order_purchase_timestamp         99441
order_estimated_delivery_date    99441
order_delivered_carrier_date     97658
order_delivered_customer_date    96476
dtype: int64

In [3]:
sold_products = pd.read_csv('../datasets/trusted/sold_products.csv')
sold_products.count()

Unnamed: 0                       104881
product_id                       104881
order_id                         104881
price                            104881
freight_value                    104881
order_purchase_timestamp         104881
order_estimated_delivery_date    104881
order_delivered_carrier_date     104881
order_delivered_customer_date    104881
product_category_name            104881
product_name_lenght              104881
product_description_lenght       104881
product_photos_qty               104881
product_weight_g                 104881
product_length_cm                104881
product_height_cm                104881
product_width_cm                 104881
dtype: int64

In [4]:
sales_by_category = all_orders[['order_id', 'customer_unique_id']].merge(sold_products[['order_id', 'product_category_name']], on='order_id')
sales_by_category.count()

order_id                 104881
customer_unique_id       104881
product_category_name    104881
dtype: int64

In [5]:
customers_with_many_cats = sales_by_category.groupby('customer_unique_id').agg('count')['product_category_name']
customers_with_many_cats = customers_with_many_cats[customers_with_many_cats > 1]
customers_with_many_cats.head()

customer_unique_id
00053a61a98854899e70ed204dd4bafe    2
000de6019bb59f34c099a907c151d855    2
000fbf0473c10fc1ab6f8d2d286ce20c    4
001147e649a7b1afd577e873841632dd    2
0015752e079902b12cd00b9b7596276b    2
Name: product_category_name, dtype: int64

In [6]:
sales_list = [list(sales_by_category[sales_by_category['customer_unique_id'] == user]['product_category_name'].unique()) for user in customers_with_many_cats.index]
sales_list

[['esporte_lazer'],
 ['cama_mesa_banho'],
 ['brinquedos', 'instrumentos_musicais'],
 ['utilidades_domesticas'],
 ['malas_acessorios'],
 ['eletronicos', 'informatica_acessorios'],
 ['moveis_sala', 'cama_mesa_banho'],
 ['papelaria'],
 ['beleza_saude'],
 ['beleza_saude'],
 ['malas_acessorios'],
 ['beleza_saude'],
 ['moveis_escritorio'],
 ['cool_stuff', 'malas_acessorios'],
 ['moveis_decoracao'],
 ['moveis_escritorio'],
 ['cama_mesa_banho'],
 ['beleza_saude'],
 ['telefonia'],
 ['ferramentas_jardim'],
 ['brinquedos'],
 ['esporte_lazer'],
 ['utilidades_domesticas'],
 ['utilidades_domesticas'],
 ['flores', 'moveis_decoracao'],
 ['esporte_lazer', 'eletronicos'],
 ['bebes', 'moveis_decoracao'],
 ['bebidas'],
 ['beleza_saude'],
 ['perfumaria'],
 ['ferramentas_jardim'],
 ['fashion_bolsas_e_acessorios'],
 ['eletrodomesticos', 'cama_mesa_banho'],
 ['cama_mesa_banho'],
 ['perfumaria'],
 ['moveis_decoracao'],
 ['moveis_decoracao'],
 ['beleza_saude'],
 ['ferramentas_jardim'],
 ['utilidades_domesticas'

Obtida a lista de compras por consumidor, agora estudamos somente aqueles que são recorrentes, isso é, possuem mais de uma compra, de modo que eles se tornem mais estatisticamente relevantes

In [7]:
recurrent_list = [(i, x) for i, x in enumerate(sales_list) if len(x) > 1]
recurrent_list

[(2, ['brinquedos', 'instrumentos_musicais']),
 (5, ['eletronicos', 'informatica_acessorios']),
 (6, ['moveis_sala', 'cama_mesa_banho']),
 (13, ['cool_stuff', 'malas_acessorios']),
 (24, ['flores', 'moveis_decoracao']),
 (25, ['esporte_lazer', 'eletronicos']),
 (26, ['bebes', 'moveis_decoracao']),
 (32, ['eletrodomesticos', 'cama_mesa_banho']),
 (39, ['utilidades_domesticas', 'cama_mesa_banho']),
 (42, ['automotivo', 'ferramentas_jardim']),
 (43, ['instrumentos_musicais', 'brinquedos']),
 (45, ['alimentos', 'esporte_lazer']),
 (46, ['beleza_saude', 'moveis_escritorio']),
 (48, ['construcao_ferramentas_iluminacao', 'moveis_decoracao']),
 (52, ['brinquedos', 'instrumentos_musicais']),
 (55, ['cama_mesa_banho', 'utilidades_domesticas']),
 (81, ['cama_mesa_banho', 'cool_stuff']),
 (95, ['informatica_acessorios', 'consoles_games']),
 (102, ['esporte_lazer', 'relogios_presentes']),
 (105, ['moveis_sala', 'cama_mesa_banho']),
 (113, ['informatica_acessorios', 'dvds_blu_ray']),
 (114, ['beleza

Criamos um *SparkContext* para acessarmos a biblioteca do *PySpark*, que possui as funcionalidades de **FPGrowth** que queremos analizar nesse estudo

In [8]:
spark = SparkSession.builder.master("local").appName("Word Count").getOrCreate()

A partir de então criamos de fato o nosso modelo de **FPGrowth**. É importante notarmos o hiperparâmetro *minSupport*, que determina qual a frequência mínima de ocorrência de um padrão para que ele seja considerado frequente. Nesse caso, ele foi configurado para 5%, isso é, qualquer padrão que apareça em ao menos 5% das compras é considerado estatisticamente relevante e portanto frequente

In [9]:
fp = FPGrowth(minSupport=0.01, minConfidence=0.15)

sdf = spark.createDataFrame(recurrent_list, ["id", "items"])
fpm = fp.fit(sdf)

In [10]:
result = fpm.freqItemsets.collect()
for fi in sorted(result, key=lambda x: x.freq, reverse=True):
    print(fi)

Row(items=['cama_mesa_banho'], freq=512)
Row(items=['moveis_decoracao'], freq=477)
Row(items=['utilidades_domesticas'], freq=290)
Row(items=['esporte_lazer'], freq=274)
Row(items=['beleza_saude'], freq=248)
Row(items=['informatica_acessorios'], freq=190)
Row(items=['ferramentas_jardim'], freq=184)
Row(items=['brinquedos'], freq=167)
Row(items=['relogios_presentes'], freq=161)
Row(items=['bebes'], freq=160)
Row(items=['cool_stuff'], freq=150)
Row(items=['moveis_decoracao', 'cama_mesa_banho'], freq=149)
Row(items=['automotivo'], freq=118)
Row(items=['telefonia'], freq=117)
Row(items=['perfumaria'], freq=114)
Row(items=['fashion_bolsas_e_acessorios'], freq=111)
Row(items=['papelaria'], freq=95)
Row(items=['eletronicos'], freq=83)
Row(items=['casa_conforto'], freq=73)
Row(items=['pet_shop'], freq=67)
Row(items=['utilidades_domesticas', 'cama_mesa_banho'], freq=59)
Row(items=['utilidades_domesticas', 'moveis_decoracao'], freq=56)
Row(items=['casa_conforto', 'cama_mesa_banho'], freq=54)
Row(

É possível perceber que a maioria dos padrões possuem somente uma categoria, o que significa que grande parte das compras as possuem em comum. Notemos, porém, a presença de `['moveis_decoracao', 'cama_mesa_banho'], freq=149`, indicando uma alta frequência de compras conjuntas desses itens.

In [11]:
fpm.associationRules.show()

+--------------------+------------------+-------------------+------------------+
|          antecedent|        consequent|         confidence|              lift|
+--------------------+------------------+-------------------+------------------+
|        [perfumaria]|    [beleza_saude]|0.21052631578947367|1.7393887945670627|
|   [cama_mesa_banho]|[moveis_decoracao]|        0.291015625|  1.25008598663522|
|             [bebes]|      [cool_stuff]|              0.175|            2.3905|
|             [bebes]|      [brinquedos]|             0.1625|1.9937874251497005|
|             [bebes]| [cama_mesa_banho]|              0.225|    0.900439453125|
|[ferramentas_jardim]|[moveis_decoracao]|0.22826086956521738|0.9805168170631664|
|     [casa_conforto]| [cama_mesa_banho]| 0.7397260273972602| 2.960348886986301|
|[utilidades_domes...|[moveis_decoracao]|0.19310344827586207|0.8294946866189546|
|[utilidades_domes...| [cama_mesa_banho]|0.20344827586206896|0.8141904633620689|
|        [brinquedos]|      